In [ ]:
# !apt-get update --fix-missing

# try:
#     import firedrake
# except ImportError:
#     !wget "https://fem-on-colab.github.io/releases/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
#     import firedrake

The available potential energy (APE) of a buoyancy field $B(\boldsymbol{x})$ of vertical extent $Z(\boldsymbol{x})$ is given by the difference of its actual potential energy and the (minimal) potential energy 
\begin{equation}
\mathbb{E}[BZ^*]  - \mathbb{E}[BZ] = \iint b \left(Z^*(b) -z \right) \; f_{BZ}(b,z) \; dbdz,
\end{equation}
where the reference state
\begin{equation}
Z^*(b) = F^{-1}_Z \circ F_B(b) = Q_Z \circ F_B(b),
\end{equation}
corresponds to an adiabatic volume preserving rearrangement of the fluid into a state that minimises PE. To calculate this quantity therefore requires an estimation of the buoyancy's PDF & CDF $f_B,F_B$ as well as the vertical cordinate's QDF $Q_Z = F_Z^{-1}$.

As an example of a buoyancy field and potential we will consider
\begin{align}
    B(x_1,x_2) &= x_2 + \epsilon \sin(2 \pi x_2) \sin(\pi x_1),\\
    Z(x_1,x_2) &= x_2,
\end{align}
such that the physical domain is given by $\Omega_X \in [-1,1] \times [0,1]$, the buoyancy domain $\Omega_B \in [-1,1]$ and that of the potential by $\Omega_Z \in [0,1]$. The parameter $\epsilon$ will be used to swich on and off a more complicated functional dependency.

To construct the buoyancy's PDF & CDF $f_B,F_B$ and the vertical cordinate's QDF $F_Z^{-1}$ we make use of the **PDF_Projector** class imported below.

In [ ]:
from PDF_Projector import *

First we set up the function spaces and domain of the buoyancy $B$

In [ ]:
ε = 1
π = np.pi

# Set the function space
ptp = FEptp(func_space_CDF = {"family":"DG","degree":1},func_space_PDF={"family":"CG","degree":1})

# Set the domain
x1,x2,b = ptp.domain(Omega_X = {'x1':(-1,1),'x2':(0,1)}, Omega_Y = {'Y':(-1,1)}, N_elements=50)

# Construct the PDF,CDF,QDF
ptp.fit(function_Y = x2 + ε*sin(2*π*x1)*sin(π*x2), quadrature_degree=200)

# Grab the CDF & PDF
F_B = ptp.F
f_B = ptp.f

Second we set up function space and domain of the potential $Z$

In [ ]:
# Set the function space
ptp = FEptp(func_space_CDF = {"family":"DG","degree":1})

# Set the domain
x1,x2,z = ptp.domain(Omega_X = {'x1':(-1,1),'x2':(0,1)}, Omega_Y = {'Y':(0,1)}, N_elements=50)

# Construct the PDF,CDF,QDF
ptp.fit(function_Y = x2, quadrature_degree=200)

# Grab the QDF
Q_Z = ptp.Q

Having obtained all necessary functions it remains to evaluate the integral to obtain the reference height $Z^*(b)$